# Road Following - 데이터 수집

도로 주힝 충돌방지 과정

1.  데이터 수집
2.  학습
3.  배포

1 데이터 수집 
    - 젯봇을 도로상에서 임의의 위치로 이동하며 도로 방향을 향하도록 카메라로 데이터 생성
    - 이떼 카메라로 수집된 데이터는 X,Y 값을 영상 이미지와 함께 저장

### Import Libraries

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### 카메라 동작 확인

JetBot의 Camera Class를 사용하여 CSI MIPI 카메라를 활성화한뒤 224x224 픽셀 이미지로 데이터를 수집. 

In [2]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0, step=0.001)

FloatSlider(value=0.0, description='y', max=1.0, min=-1.0, step=0.001)

### 데이터 수집

In [3]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')


button_layout = widgets.Layout(width='128px', height='64px')
button = widgets.Button(description='add', button_style='danger', layout=button_layout)
count = widgets.IntText(layout=button_layout, value=len(os.listdir(DATASET_DIR)))

display(widgets.HBox([count, button]))

Directories not created becasue they already exist


In [4]:
widgets.jsdlink((x_slider, 'value'), (x_slider, 'value'))
widgets.jsdlink((y_slider, 'value'), (y_slider, 'value'))

DirectionalLink(source=(FloatSlider(value=0.0, description='y', max=1.0, min=-1.0, step=0.001), 'value'), targ…

In [5]:
def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot():
    if True or change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

# controller.buttons[13].observe(save_snapshot, names='value')
button.on_click(lambda x: save_snapshot())

display(widgets.HBox([
    count,
    button
]))

### 수집 데이터 저장

In [ ]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q road_following_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}